In [1]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit import *
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import TwoLocal, UniformDistribution
from qiskit.quantum_info import Statevector
from qiskit_machine_learning.algorithms import NumPyDiscriminator, QGAN
from qiskit.aqua.operators.gradients import Gradient
from qiskit.aqua import *

In [2]:
bat= 100
epo= 100

In [6]:
def gener(x):
    count = 0
    samps = []
    while(count < x):
        a = np.random.randint(0,4)
        b = np.random.randint(0,4)
        if a < b: #we can chooose to leave certain samples out here to experiment with the results
            samps.append([int(a/2),a%2,int(b/2),b%2])
            count += 1
    return samps
        

In [7]:
samps= np.array(gener(2000))

In [8]:
samps

array([[0, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 1, 1, 0],
       ...,
       [1, 0, 1, 1],
       [0, 1, 1, 0],
       [0, 0, 1, 1]])

In [9]:
qg =QGAN(samps,bounds=np.array([[0,1],[0,1],[0,1],[0,1]]),num_qubits=[1,1,1,1],batch_size=bat,num_epochs=epo)

In [13]:
quantum_instance = QuantumInstance(backend=BasicAer.get_backend('statevector_simulator'),shots=10,
                                   seed_transpiler=71, seed_simulator=71)

/Users/ardanacar/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_class('aqua.QuantumInstance',


In [14]:
ans = TwoLocal(4, 'ry', 'cz', entanglement='circular', reps=1)

In [15]:
ini = UniformDistribution(4)

<ipython-input-15-4d29978329e3>:1: DeprecationWarning: `UniformDistribution` is deprecated as of version 0.17.0 and will be removed no earlier than 3 months after the release date. It moved to qiskit_finance.circuit.library.UniformDistribution.
  ini = UniformDistribution(4)


In [16]:
gen = ans.compose(ini, front = True)

In [17]:
gen.draw()

┌───┐┌──────────┐      ┌──────────┐                        
q_0: ┤ H ├┤ RY(θ[0]) ├─■──■─┤ RY(θ[4]) ├────────────────────────
     ├───┤├──────────┤ │  │ └──────────┘┌──────────┐            
q_1: ┤ H ├┤ RY(θ[1]) ├─┼──■──────■──────┤ RY(θ[5]) ├────────────
     ├───┤├──────────┤ │         │      └──────────┘┌──────────┐
q_2: ┤ H ├┤ RY(θ[2]) ├─┼─────────■───────────■──────┤ RY(θ[6]) ├
     ├───┤├──────────┤ │                     │      ├──────────┤
q_3: ┤ H ├┤ RY(θ[3]) ├─■─────────────────────■──────┤ RY(θ[7]) ├
     └───┘└──────────┘                              └──────────┘

In [18]:
inip = np.random.rand(8)

In [19]:
qg.set_generator(generator_circuit=gen,generator_init_params=inip)

In [20]:
qg.generator.optimizer.set_options(lr=0.006)

In [21]:
qg._generator._free_parameters = sorted(gen.parameters, key=lambda p: p.name)
disc = NumPyDiscriminator(4)
qg.set_discriminator(disc)

In [22]:
r = qg.run(quantum_instance)

KeyboardInterrupt: 

In [ ]:
qg.generator.get_output(quantum_instance, shots=10)